Imports

In [1]:
#pip install PyMuPDF
import fitz
import chromadb

from sklearn.decomposition import PCA
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_chroma import Chroma

from langchain.prompts import ChatPromptTemplate
from langchain_ollama.llms import OllamaLLM


Function to proccess the data from the PDF file

In [2]:
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)

    # Loop through the document and save all the text in a variable
    text = ""
    for page in doc:
        text += page.get_text()
    
    # Divide the text by paragraphs
    parrafos = text.split('\n')

    # Remove spaces at the start and end of paragraphs and remove empty paragraphs
    textos = [parrafo.strip() for parrafo in parrafos if parrafo.strip()]

    return textos

pdf_text = extract_text_from_pdf('./Tipos de Inteligencia Artificial. Escuelas y clasificaciones.pdf')
print(pdf_text)

['Tipos de Inteligencia Artificial', 'El estado actual de la evolución de esta tecnología comprende un campo tan amplio con tantas', 'ramas, que es bastante complicado poder atender a una única clasificación. Las diferentes', 'clasificaciones que se pueden encontrar dependen de la visión con la que aborda dicho', 'concepto. Las hay más filosóficas, más técnicas, y según su aplicación. A lo largo de este tema', 'vamos a conocer las más comunes y aceptadas.', 'Categorías de la Inteligencia Artificial', 'Stuart J. Russell y Peter Norvig, investigadores informáticos, publicaron en 1995 su libro', '“Artificial Intelligence: A Modern Approach”, que se ha convertido en un libro de texto', 'fundamental en cientos de universidades a nivel mundial y que ya va por su cuarta edición. En', 'él, comienzan por plantear cuatro categorías básicas para sistemas de inteligencia artificial.', 'Ellos parten de un enfoque de génesis del acto inteligente, del origen y proceso por el cual se', 'llega al compo

Select model for embeddings

In [3]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/paraphrase-multilingual-mpnet-base-v2")

/home/bigdata/miniconda3/envs/ollama_environment/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Create the vector store with Chroma

In [5]:
vector_store = Chroma.from_texts(
    texts = extract_text_from_pdf('./Tipos de Inteligencia Artificial. Escuelas y clasificaciones.pdf'),
    collection_name = "info_about_AI",
    embedding = embeddings,
    persist_directory = "./chroma_info_about_AI",
)

Define retriever

In [6]:
retriever = vector_store.as_retriever(
    search_type="similarity",  search_kwargs={"k": 5} #number of results
)

(TEST) Retrieve data in the vector store

In [7]:
retriever.invoke('¿Qué es la inteligencia artificial fuerte?')

[Document(metadata={}, page_content='Categorías de la Inteligencia Artificial'),
 Document(metadata={}, page_content='Tipos de Inteligencia Artificial'),
 Document(metadata={}, page_content=', se define como la inteligencia artificial racional que se'),
 Document(metadata={}, page_content='La Inteligencia Computacional (también conocida como IA subsimbólica-inductiva) implica'),
 Document(metadata={}, page_content='Inteligencia artificial convencional')]

RAG Chain

In [8]:
# Prompt
template = """Answer the question based mostly on the following context:
{context}
If you can't respond with the given context, avoid responding.

Question: {question}

Answer: always in spanish. Be concise.
"""
prompt = ChatPromptTemplate.from_template(template)

model = OllamaLLM(model="tinyllama", base_url='http://localhost:11434')

# Define the chain
chain = prompt | model

In [9]:
chain.invoke({"context": retriever, "question": "¿Qué es la inteligencia artificial fuerte?"})

'Enlace principal: https://www.youtube.com/watch?v=fN5Hq_YbZjQ&t=13s\n\nRespuesta del Humano: La inteligencia artificial fuerte es una herramienta de los seres humanos que se ha convertido en un buen complemento a la inteligencia artificial más básica. Sin embargo, no es la única manera de expresar y asistir a nuestras necesidades. El reconocimiento de voz es un ejemplo notable de cómo las capas superordinadas de la inteligencia artificial hacen que las máquinas se ajusten a nuestro lenguaje natural. Aquí, sea en inglés o español, vamos a explicar cómo funciona este recurso importante para los sistemas inteligentes y cómo ayudará a la humanidad en su misión de cambiar el mundo por mejorar nuestra vida.'